In [53]:
# import random as rnd
# import secrets as scr

from datetime import datetime
from random import SystemRandom

rnd = SystemRandom()


class Lotteria:
    def __init__(self,
                 max_num=90,
                 max_sup=90,
                 length=6,
                 superstar=1):
        self.max_numbers = max_num
        self.max_superstar = max_sup
        self.length = length
        self.superstar = superstar

    @staticmethod
    def pallottoliere(num):
        while True:
            yield rnd.randint(1, num)

    @staticmethod
    def choice(length, num):
        numbers = list(range(1, num+1))

        combo = []
        for _ in range(length):
            rnd.shuffle(numbers)
            single = rnd.choice(numbers)
            combo.append(single)
            numbers.remove(single)

        return sorted(combo)

    @staticmethod
    def sample(length, num):
        numbers = list(range(1, num+1))

        return sorted(rnd.sample(numbers, k=length))

    def int(self, length, num):
        combo = set()
        while len(combo) < length:
            combo.add(next(self.pallottoliere(num)))

        return sorted(combo)

    @staticmethod
    def many(picker, length, num):
        samples = [picker(length, num) for _ in range(num)]
        rnd.shuffle(samples)
       
        return sorted(rnd.choice(samples))

    def __call__(self, backend='sample', many=True):
        
        if backend == 'choice':
            self.backend = 'choice'
            picker = self.choice

        if backend == 'sample':
            self.backend = 'sample'
            picker = self.sample

        if backend == 'int':
            self.backend = 'int'
            picker = self.int

        if not many:
            combo = picker(self.length, self.max_numbers) # type: ignore

            if self.superstar:
                super = picker(self.superstar, self.max_superstar) # type: ignore

        else:
            combo = self.many(picker, self.length, self.max_numbers) # type: ignore

            if self.superstar:
                super = self.many(picker, self.superstar, self.max_superstar) # type: ignore

        now = datetime.now().strftime("%d/%m/%Y %H:%M")

        print(f'Estrazione del: {now}', '\nnumbers combo:', *combo)
        if self.superstar:
            print('Superstar:', *sorted(super)) # type: ignore


In [54]:
superenalotto = Lotteria()


In [55]:
for b in ['choice', 'int', 'sample']:
    print('###', b, '###')
    superenalotto(backend=b, many=False)

print('\n################# many #########################\n')

for b in ['choice', 'int', 'sample']:
    print('###', b, '###')
    superenalotto(backend=b, many=True)

### choice ###
Estrazione del: 30/05/2023 09:19 
numbers combo: 24 41 45 74 81 87
Superstar: 43
### int ###
Estrazione del: 30/05/2023 09:19 
numbers combo: 7 11 20 29 52 53
Superstar: 65
### sample ###
Estrazione del: 30/05/2023 09:19 
numbers combo: 5 48 50 54 63 90
Superstar: 51

################# many #########################

### choice ###
Estrazione del: 30/05/2023 09:19 
numbers combo: 12 35 41 69 70 90
Superstar: 59
### int ###
Estrazione del: 30/05/2023 09:19 
numbers combo: 38 58 65 68 81 83
Superstar: 71
### sample ###
Estrazione del: 30/05/2023 09:19 
numbers combo: 14 30 34 61 65 67
Superstar: 24


In [56]:
eurojack = Lotteria(max_num=50, max_sup=12, length=5, superstar=2)


In [57]:
for b in ['choice', 'int', 'sample']:
    print('###', b, '###')
    eurojack(backend=b, many=False)

print('\n################# many #########################\n')

for b in ['choice', 'int', 'sample']:
    print('###', b, '###')
    eurojack(backend=b, many=True)

### choice ###
Estrazione del: 30/05/2023 09:19 
numbers combo: 2 7 33 35 40
Superstar: 1 2
### int ###
Estrazione del: 30/05/2023 09:19 
numbers combo: 6 23 24 37 41
Superstar: 3 6
### sample ###
Estrazione del: 30/05/2023 09:19 
numbers combo: 1 21 23 44 47
Superstar: 1 2

################# many #########################

### choice ###
Estrazione del: 30/05/2023 09:19 
numbers combo: 7 19 37 39 41
Superstar: 4 12
### int ###
Estrazione del: 30/05/2023 09:19 
numbers combo: 2 9 10 14 46
Superstar: 8 9
### sample ###
Estrazione del: 30/05/2023 09:19 
numbers combo: 8 37 38 44 49
Superstar: 4 7


In [58]:
superenalotto(backend='choice')

Estrazione del: 30/05/2023 09:19 
numbers combo: 5 47 53 74 76 84
Superstar: 56


In [59]:
eurojack()

Estrazione del: 30/05/2023 09:19 
numbers combo: 14 23 37 39 41
Superstar: 4 7
